# Variational quantum eigensolvers for solving this matrix:   
\begin{bmatrix} 
1 & 0& 0&0\\ 
0& 0& -1&0\\
0 & -1 &0&0\\
0&0&0&1\\
\end{bmatrix} 


<b>We do this on 3 steps :</b>
<ol>
<li>Constructing the anzatz</li>
<li>Hamiltonian</li>
<li>Do the measurements</li>
</ol> 

## I am going to use Qiskit & Scipy

In [4]:
import numpy as np; pi = np.pi
from qiskit import QuantumCircuit, Aer, execute
from scipy.optimize import minimize
from qiskit.visualization import plot_histogram
from qiskit.circuit.library.standard_gates import U2Gate
from copy import deepcopy as make_copy

# First preparing the anzatz: 
using the hint that in the <b>screening Task paper</b> the anzatz is consisting of **-**$R_{x}$ ,**Cnot** and **H** **-** Rotation X, Controlled not and Hadmard gates with depth of the circuit is equal to   **D=2**.


In [5]:
def prepare_anz_circuit(depth, angle1):

    """
    This function accepts two arguments it takes the depth and the angle  arguments and performs
    some operations. First it create the quantum circuit. second it perform the anzatz constructions
    by applying rotation x in first qubit and then applying controlled not on the two qubits then 
    we apply the hadmard gate on first qubit. We performs this operations in the range of the depth 
    that we passed.
    """
    anz_circ = QuantumCircuit(depth)
    for i in range(depth): 
        anz_circ.rx(angle1[0], 0) 
        anz_circ.cx(0, 1) 
        anz_circ.h(0)
        
    return anz_circ 



# Second constructing the Hamiltonian: 
## after the calculations I find the Hamiltonian is equal to: 
### H=0.5* $Z\bigotimes Z $- 0.5* $X\bigotimes X$+ 0.5* $I\bigotimes I$ - 0.5* $Y\bigotimes Y$

# Third Doing the measurements: 


### first we define a measurements expression for **Z** measurements than we calculate the expectation value for it $\langle ZZ \rangle$

In [6]:
simulator = Aer.get_backend('qasm_simulator')

In [7]:
def measure_zz_circuit(angle2):
    
    """
    This function does  the zz measurements.It takes one arguments that one arguments the angle and 
    passes it to the prepare anzatz function then we make the copy if the circuit then we measure it.
    """
    hets_circuit =prepare_anz_circuit(2,angle2)
    zz_meas = make_copy(hets_circuit)
    zz_meas.measure_all()
    return zz_meas

In [8]:
def measure_zz(angle2, num_shots = 10000):
      
        
        """
       This function calculates the expectation value for <ZZ>. It takes two arguments the angle and 
       the number of shots and then after doing the measurements we get the expectation value for <ZZ>.
        """
        zz_meas = measure_zz_circuit(angle2)
        
        result = execute(zz_meas, backend = simulator, shots = num_shots).result()
        counts = result.get_counts(zz_meas)

        if '00' not in counts:
            counts['00'] = 0
        if '01' not in counts:
            counts['01'] = 0
        if '10' not in counts:
            counts['10'] = 0
        if '11' not in counts:
            counts['11'] = 0 

        total_counts = counts['00'] + counts['11'] + counts['01'] + counts['10']
        zz = counts['00'] + counts['11'] - counts['01'] - counts['10']
        zz = zz / total_counts
    
        return zz


    
    

### First we define a measurements expression for **X** measurements than we calculate the expectation value for it $\langle XX \rangle$


#### I want to demonstrate why i use Hadmard gate before doing the measurements: 
we need to measure in computational basis**$| 0 \rangle$** and **$| 1\rangle$** so we need to change the basis of **X** which is **$|i\rangle$** and **$|-i\rangle$** so we need unitary transformation that do this so After the calculation it turns out that it we need to perform hadmard transformation so that we can do our the measurements in computational basis.


In [9]:

def measure_xx_circuit(angle2):
    """
    This function does the xx measurements.It takes one arguments that one arguments the angle and 
    passes it to the prepare anzatz function then we make the copy if the circuit then we measure it.
   """
    hets_circuit =prepare_anz_circuit(2,angle2)
    xx_meas = make_copy(hets_circuit)
        
    xx_meas.h(0)
    xx_meas.h(1)
    xx_meas.measure_all()
    
    return xx_meas


In [10]:

def measure_xx(angle2, num_shots = 10000):
    """
    This function calculates the expectation value for <ZZ>. It takes two arguments the angle and 
    the number of shots and then after doing the measurements we get the expectation value for <ZZ>.
    """
    xx_meas = measure_xx_circuit(angle2)
    
    result = execute(xx_meas, backend = simulator, shots = num_shots).result()
    counts = result.get_counts(xx_meas)

    if '00' not in counts:
        counts['00'] = 0
    if '01' not in counts:
        counts['01'] = 0
    if '10' not in counts:
        counts['10'] = 0
    if '11' not in counts:
        counts['11'] = 0

    total_counts = counts['00'] + counts['11'] + counts['01'] + counts['10']
    xx = counts['00'] + counts['11'] - counts['01'] - counts['10']
    xx = xx / total_counts
    
    return xx
    

### First we define a measurements expression for **Y** measurements than we calculate the expectation value for it $\langle YY \rangle$.

#### I want to demonstrate why i use Y gate before doing the measurements: 
we need to measure in computational basis**$| 0 \rangle$** and **$| 1\rangle$** so we need to change the basis of **X** which is **$|+\rangle$** and **$| -\rangle$** so we need unitary transformation that do this so After the calculation it turns out that it we need to perform Y transformation so that we can do our the measurements in computational basis. This **Y**  transformation is in the form of 
\begin{equation*}
Y_{gate}= \frac{1}{\sqrt{2}}
\begin{bmatrix} 
1&-i\\ 
1&i
\end{bmatrix}
\end{equation*}
so I use the **u gate** that is in qiskit which in form with **$\phi$ =pi/2** and **$\lambda$=0**
\begin{equation*}
u_{gate}(\phi,\lambda)= \frac{1}{\sqrt{2}}
\begin{bmatrix} 
1&-e^{i\lambda}\\ 
e^{\phi\lambda}&e^{i(\phi+\lambda)}
\end{bmatrix}
\end{equation*}

In [11]:
def measure_yy_circuit(angle1):
  
    """
    This function does the yy measurements.It takes one arguments that one arguments the angle and 
    passes it to the prepare anzatz function then we make the copy if the circuit then we measure it.
    """

    anz_circuit = prepare_anz_circuit(2,angle1)
    yy_meas= make_copy(anz_circuit)

    
    yy_meas.u2(0, np.pi/2, 0)
    yy_meas.u2(0, np.pi/2, 1)
    yy_meas.measure_all()
    

    return yy_meas


In [12]:

 def measure_yy(angle1, num_shots = 10000):
    """
    This function calculates the expectation value for <YY>. It takes two arguments the angle and 
    the number of shots and then after doing the measurements we get the expectation value for <YY>.
    """
    yy_meas = measure_yy_circuit(angle1)
    
    result = execute(yy_meas, backend = simulator, shots = num_shots).result()
    counts = result.get_counts(yy_meas)

    if '00' not in counts:
        counts['00'] = 0
    if '01' not in counts:
        counts['01'] = 0
    if '10' not in counts:
        counts['10'] = 0
    if '11' not in counts:
        counts['11'] = 0 

    total_counts = counts['00'] + counts['11'] + counts['01'] + counts['10']
    yy = counts['00'] + counts['11'] - counts['01'] - counts['10']
    yy = yy / total_counts
    
    return yy

In [13]:
def get_ev(angle2, num_shots = 10000):
    """
    This  function calculate the eigenvalue for our matrix. It takes 2 parameters the angle and num 
    of shots and then call the measurements function and then calculate the energy using the
    Hamiltonian that we calculate and done.
    """
    zz= measure_zz(angle2, num_shots = num_shots)
    xx= measure_xx(angle2, num_shots = num_shots)
    yy= measure_yy(angle2, num_shots = num_shots)

    energy =0.5*zz-0.5*xx+0.5-0.5*yy
    return energy

In [14]:
arr = np.array([np.pi, np.pi])
tol = 1e-3 # tolerance for optimization precision.
l_eigen_value= minimize(get_ev, arr, method="Powell", tol=tol)
print('The estimated ground state energy from VQE algorithm is: {}'.format(l_eigen_value.fun))


The estimated ground state energy from VQE algorithm is: -1.0
